In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [3]:
import netCDF4 as nc
import sys
sys.path.append('/data/tjarniko/mocsy')
import mocsy
import numpy as np

import gsw
import arrow
import glob
import cmocean as cm
import matplotlib.pyplot as plt

from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)

%matplotlib inline

## Purpose of notebook

Here I'm going to take the S, and T we've found in, eg, carp_timeseries_2013.nc, and convert them into the format you want.




In [4]:
t_nc = nc.Dataset('./EXTRACTED_timeseries/carp_timeseries_2016.nc')
t_str = 'TS_timeseries_2016.nc'

In [5]:
def conv_TS(tsal, ttemp):

    size_box = np.shape(tsal)
    size_0 = size_box[0]
    size_1= size_box[1]

    tsra = np.ravel(tsal)
    ttera = np.ravel(ttemp)
    tzero = np.zeros_like(tsra)
       
    tsra_psu = tsra*35/35.16504
    ttera_is = gsw.t_from_CT(tsra,ttera,tzero)


    tsra_psuR = tsra_psu.reshape(size_0,size_1)
    ttera_isR = ttera_is.reshape(size_0,size_1)
    
    return tsra_psuR, ttera_isR

#### TJSJ let's make a dict in which to store the OmA and pH timeseries

In [6]:
PLACES_withdat = {
'Deep Bay': {
'NEMO grid ji': (599, 126),
    'T_timeseries':np.zeros([40,365]),
    'S_timeseries':np.zeros([40,365]),

},
'Southern Baynes': {
'NEMO grid ji': (602, 127),
    'T_timeseries':np.zeros([40,365]),
    'S_timeseries':np.zeros([40,365]),
},

'Northern Baynes': {
'NEMO grid ji': (646, 127),
    'T_timeseries':np.zeros([40,365]),
    'S_timeseries':np.zeros([40,365]),
    },

'Fanny Bay': {
'NEMO grid ji': (614, 120),
    'T_timeseries':np.zeros([40,365]),
    'S_timeseries':np.zeros([40,365]),
    },

'Maple Bay': {
'NEMO grid ji': (392, 213),
    'T_timeseries':np.zeros([40,365]),
    'S_timeseries':np.zeros([40,365]),
},

'Salt Spring': {
'NEMO grid ji': (386, 218),
    'T_timeseries':np.zeros([40,365]),
    'S_timeseries':np.zeros([40,365]),
},

'Nanoose Bay': {
'NEMO grid ji': (517, 190),
    'T_timeseries':np.zeros([40,365]),
    'S_timeseries':np.zeros([40,365]),
},

'Lasqueti Island': {
'NEMO grid ji': (586, 195),
    'T_timeseries':np.zeros([40,365]),
    'S_timeseries':np.zeros([40,365]),
},

'Main SoG': {
'NEMO grid ji': (450, 253),
    'T_timeseries':np.zeros([40,365]),
    'S_timeseries':np.zeros([40,365]),

},

'Cortes/Marina': {
'NEMO grid ji': (732, 157),
    'T_timeseries':np.zeros([40,365]),
    'S_timeseries':np.zeros([40,365]),
    },

'Lund/Desolation Sound': {
'NEMO grid ji': (702, 187),
    'T_timeseries':np.zeros([40,365]),
    'S_timeseries':np.zeros([40,365]),
    },

'Mouth of Okeover': {
'NEMO grid ji': (726, 192),
    'T_timeseries':np.zeros([40,365]),
    'S_timeseries':np.zeros([40,365]),
    },

}

list_places = ['Deep Bay', 'Southern Baynes', 'Northern Baynes', \
               'Fanny Bay', 'Maple Bay', 'Salt Spring', 'Nanoose Bay',\
               'Lasqueti Island', 'Main SoG', 'Cortes/Marina', \
               'Lund/Desolation Sound', 'Mouth of Okeover']


#### TJSJ let's open the .nc we made in part B, extract the DIC, TA, S, T, variables and run mocsy on them to get pH and Omega A, then store these in the dictionary we just made

In [7]:
for i in range(0,len(list_places)):
    stn = list_places[i]

    TEMP = t_nc[stn]['T_timeseries'][:,:]
    SAL = t_nc[stn]['S_timeseries'][:,:]

    SAL_psu, TEMP_is = conv_TS(SAL, TEMP)
    
    PLACES_withdat[stn]['S_timeseries'] = SAL_psu
    PLACES_withdat[stn]['T_timeseries'] = TEMP_is



In [8]:


f = nc.Dataset(t_str, 'w')
g = f.createGroup('Deep Bay')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('T_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Deep Bay']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Deep Bay']['S_timeseries'][:]

g = f.createGroup('Southern Baynes')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('T_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Southern Baynes']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Southern Baynes']['S_timeseries'][:]

g = f.createGroup('Northern Baynes')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('T_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Northern Baynes']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Northern Baynes']['S_timeseries'][:]

g = f.createGroup('Fanny Bay')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('T_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Fanny Bay']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Fanny Bay']['S_timeseries'][:]

g = f.createGroup('Maple Bay')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('T_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Maple Bay']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Maple Bay']['S_timeseries'][:]

g = f.createGroup('Salt Spring')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('T_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Salt Spring']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Salt Spring']['S_timeseries'][:]

g = f.createGroup('Nanoose Bay')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('T_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Nanoose Bay']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Nanoose Bay']['S_timeseries'][:]

g = f.createGroup('Lasqueti Island')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('T_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Lasqueti Island']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Lasqueti Island']['S_timeseries'][:]

g = f.createGroup('Main SoG')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('T_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Main SoG']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Main SoG']['S_timeseries'][:]

g = f.createGroup('Cortes/Marina')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('T_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Cortes/Marina']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Cortes/Marina']['S_timeseries'][:]

g = f.createGroup('Lund/Desolation Sound')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('T_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Lund/Desolation Sound']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Lund/Desolation Sound']['S_timeseries'][:]

g = f.createGroup('Mouth of Okeover')
g.createDimension('depths', 40)
g.createDimension('days',365)
ts = g.createVariable('T_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Mouth of Okeover']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths','days'))
ts[:] = PLACES_withdat['Mouth of Okeover']['S_timeseries'][:]

f.close()

In [9]:
print('done, yo')

done, yo
